In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!pip3 install glove_python

In [2]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [18]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
from glove import Glove,Corpus

In [26]:
import re
from sklearn.preprocessing import MultiLabelBinarizer

limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [5]:
len(labels)

78

In [6]:
data.head()

,body,tag
0,"['قسمت','ادامه','ترفندها','پیشرفته','jQuery','...","[' نرم افزار', ' کد', 'html', ' css', ' c']"
1,"['زبان','برنامه','نویسی','خواندن','درک','دشوار...","[' آموزش برنامه نویسی', ' برنامه نویسی اندروید..."
2,"['کتاب','جامعه','خودمانی','میخواندم','صفحه','ا...",['کتاب']
3,"['افتادهباشید','اینستاگرام','کامپیوتر','باشیدد...",['اینستاگرام']
4,"['روزها','بازار','استارتاپها','داغ','کشورها','...",['استارتاپ']


In [30]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [ ]:
# vocab = []
# for d in X:
#     for w in d:
#         if w not in vocab: vocab.append(w)
# vocab = sorted(vocab)
# vocab_size = len(vocab)
# print('vocab examples:', vocab[:10])
# pandas.DataFrame(vocab).to_csv("vocab.csv",header=None)

In [31]:
import re

freq_dist = pandas.read_csv('../Data/FreqDist_sorted_'+str(limit_number)+'.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word = re.sub(r"[\\u200c]","",item.replace(" ",""))
    if word!=' ' and word!= None and word!='':
      vocab.append(word)
  except:
    # print(word)
    pass
  

In [32]:
len(vocab) , len(freq_dist)

(186516, 186522)

In [33]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [34]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}

vocab size 186516


In [36]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
        try:
          
          vec = [w2i[w] for w in d if w in w2i]
          pad_len = max(0, sentence_maxlen - len(vec))
          vec += [0] * pad_len
          vec_data.append(vec)
          # print('-----------------------')
        except:
          print(type(d), d==None)
      
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape)

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300)


In [39]:
corpus = Corpus() 
#Training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(X, window=10)

glove = Glove(no_components=5, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [50]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file='glove.model', word2vec_output_file="gensim_glove_vectors.txt")    
from gensim.models.keyedvectors import KeyedVectors
glove_embd_w = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

UnicodeDecodeError: ignored

In [38]:
# def load_glove_weights(glove_dir, embd_dim, vocab_size, word_index):
#     embeddings_index = {}
#     f = open(os.path.join(glove_dir, 'glove.6B.' + str(embd_dim) + 'd.txt'))
#     for line in f:
#         values = line.split()
#         word = values[0]
#         coefs = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = coefs
#     f.close()

#     print('Found %s word vectors.' % len(embeddings_index)) 
#     embedding_matrix = np.zeros((vocab_size, embd_dim))
#     print('embed_matrix.shape', embedding_matrix.shape)
#     for word, i in word_index.items():
#         embedding_vector = embeddings_index.get(word)
#         if embedding_vector is not None:
#             # words not found in embedding index will be all-zeros.
#             embedding_matrix[i] = embedding_vector

#     return embedding_matrix

# embd_dim = 300
# glove_embd_w = load_glove_weights('./dataset', embd_dim, vocab_size, w2i)
###########################
# import gensim

# embd_dim = 300
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# embed_model=gensim.models.Word2Vec.load('word2vec_model')
############################




Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28


KeyboardInterrupt: ignored

In [45]:
embd_dim=300
glove_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  # if word in embed_model.wv.vocab:
        embedding_vector =glove[word]
  
        # words not found in embedding index will be all-zeros.
        glove_embd_w[i] = embedding_vector

TypeError: ignored

In [ ]:
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[glove_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn
    cnn = Conv2D(1, 
                 kernel_size=(3, sentence_maxlen),
                 activation='relu')(embd_sentence)
    cnn =  Lambda(lambda x: K.sum(x, axis=3))(cnn)
    cnn = MaxPooling1D(3)(cnn)
    cnn = Lambda(lambda x: K.sum(x, axis=2))(cnn)
    out = Dense(len(labels), activation='sigmoid')(cnn)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=["accuracy", "binary_accuracy",
    "categorical_accuracy",]) 
    return model

model = Net(vocab_size, embd_dim, sentence_maxlen, glove_embd_w)
print(model.summary())


Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 4425)]            0         
_________________________________________________________________
shared_embd (Embedding)      (None, 4425, 300)         55954800  
_________________________________________________________________
permute_2 (Permute)          (None, 300, 4425)         0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 300, 4425, 1)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 298, 1, 1)         13276     
_________________________________________________________________
lambda_7 (Lambda)            (None, 298, 1)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 99, 1) 

In [ ]:
model.fit(X_train, y_train,
            batch_size=32,
            epochs=5,
            validation_data=(X_val, y_val)
        )

Epoch 1/5
405/405 [==============================] - 472s 1s/step - loss: 0.5487 - accuracy: 0.0169 - categorical_accuracy: 0.0169 - val_loss: 0.5098 - val_accuracy: 0.0039 - val_categorical_accuracy: 0.0039
Epoch 2/5
405/405 [==============================] - 463s 1s/step - loss: 0.4770 - accuracy: 0.0036 - categorical_accuracy: 0.0036 - val_loss: 0.4466 - val_accuracy: 0.0039 - val_categorical_accuracy: 0.0039
Epoch 3/5
405/405 [==============================] - 460s 1s/step - loss: 0.4208 - accuracy: 0.0036 - categorical_accuracy: 0.0036 - val_loss: 0.3968 - val_accuracy: 0.0039 - val_categorical_accuracy: 0.0039
Epoch 4/5
405/405 [==============================] - 461s 1s/step - loss: 0.3762 - accuracy: 0.0036 - categorical_accuracy: 0.0036 - val_loss: 0.3570 - val_accuracy: 0.0039 - val_categorical_accuracy: 0.0039
Epoch 5/5
405/405 [==============================] - 458s 1s/step - loss: 0.3404 - accuracy: 0.0036 - categorical_accuracy: 0.0036 - val_loss: 0.3249 - val_accuracy: 0.

In [ ]:
model.save('cnn_model_'+str(limit_number)+'_accuracy_categoricalaccuracy_.h5')

# from keras.models import load_model
# model = load_model('cnn_model.h5')

Evaluation

In [ ]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [ ]:
print(pred[0])#example

[0.26067388 0.26465452 0.26532543 0.25120264 0.25749493 0.25086457
 0.26477438 0.25336024 0.25230414 0.26047125 0.2650743  0.25736323
 0.26310614 0.2512095  0.25120574 0.2645912  0.2627194  0.26353997
 0.260335   0.2559106  0.2621843  0.26167887 0.2528109  0.26419723
 0.2569863  0.26507106 0.264779   0.26402062 0.26212293 0.2640186
 0.25563282 0.25845373 0.2621318  0.2650153  0.25685564 0.2645614
 0.26466    0.25580838 0.25476915 0.25268584 0.2652459  0.2607805
 0.2579322  0.26467067 0.26471263 0.25154263 0.25319153 0.2646209
 0.26456147 0.25665778 0.25230342 0.2518284  0.25381112 0.2581999
 0.26293337 0.25329822 0.25526166 0.25463665 0.26458776 0.26419026
 0.26517963 0.254416   0.26569057 0.26478577 0.2578693  0.26475787
 0.25303414 0.25950643 0.26462162 0.2648151  0.26297373 0.26481962
 0.26045814 0.25309712 0.25223267 0.2641736  0.25619102 0.26191378]


In [ ]:
y_pred=[]
for l in pred:
  temp=[]
  for value in l:
    if value>=0.5:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       116
           1       0.00      0.00      0.00       150
           2       0.00      0.00      0.00       146
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        78
           5       0.00      0.00      0.00        15
           6       0.00      0.00      0.00       163
           7       0.00      0.00      0.00        32
           8       0.00      0.00      0.00        26
           9       0.00      0.00      0.00       104
          10       0.00      0.00      0.00       152
          11       0.00      0.00      0.00        69
          12       0.00      0.00      0.00       138
          13       0.00      0.00      0.00        12
          14       0.00      0.00      0.00        15
          15       0.00      0.00      0.00       148
          16       0.00      0.00      0.00       139
          17       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
